In [1]:
def getKeywordSequence(keyword):
  sequence = []
  for pos, ch in enumerate(keyword):
    previousLetters = keyword[:pos]
    newNumber = 1
    for previousPos, previousCh in enumerate(previousLetters):
      if previousCh > ch:
        sequence[previousPos] += 1
      else:
        newNumber += 1
    sequence.append(newNumber)
  return sequence
    
def createEncMatrix(width, message):
  r = 0
  c = 0
  matrix = [[]]
  for pos, ch in enumerate(message):
    matrix[r].append(ch)
    c += 1
    if c >= width:
      c = 0
      r += 1
      matrix.append([])

  return matrix
    
def encrypt(message, keyword):
  matrix = createEncMatrix(len(keyword), message)
  keywordSequence = getKeywordSequence(keyword)

  ciphertext= "";
  for num in range(len(keywordSequence)):
    pos = keywordSequence.index(num+1)
    for row in range(len(matrix)):
      if len(matrix[row]) > pos:
        ciphertext += matrix[row][pos]
  return ciphertext
    
def shift_rows(matrix):
    num_rows = len(matrix)
    for row in range(num_rows):
        matrix[row] = matrix[row][row:] + matrix[row][:row]
#S-DES can only transfer 8 bits per block, so split the string into 8-bit blocks

def split_string_into_8_byte_chunks_with_padding(input_string):
    chunks = []

    # Iterate through the input string in steps of 1 byte
    for i in range(0, len(input_string), 1):
        chunk = input_string[i:i+1]  # Get the next 1 byte
        # Check if the chunk is less than 1 byte
        if len(chunk) < 1:
            # Pad the chunk with null bytes (0x00) to make it 8 bytes long
            chunk = chunk.ljust(1, ' ')  # Using bytes for padding
        chunks.append(chunk)  # Add the chunk to the list

    return chunks
    
def shift_left(k, nth_shifts):
    s = ""
    for i in range(nth_shifts):
        for j in range(1, len(k)):
            s = s + k[j]
        s = s + k[0]
        k = s
        s = ""
    return k

def bin2hex(s):
    mp = {"0000": '0',
          "0001": '1',
          "0010": '2',
          "0011": '3',
          "0100": '4',
          "0101": '5',
          "0110": '6',
          "0111": '7',
          "1000": '8',
          "1001": '9',
          "1010": 'A',
          "1011": 'B',
          "1100": 'C',
          "1101": 'D',
          "1110": 'E',
          "1111": 'F'}
    hex = ""
    for i in range(0, len(s), 4):
        ch = ""
        ch = ch + s[i]
        ch = ch + s[i + 1]
        ch = ch + s[i + 2]
        ch = ch + s[i + 3]
        hex = hex + mp[ch]
 
    return hex
    
def hex2bin(s):
    mp = {'0': "0000",
          '1': "0001",
          '2': "0010",
          '3': "0011",
          '4': "0100",
          '5': "0101",
          '6': "0110",
          '7': "0111",
          '8': "1000",
          '9': "1001",
          'A': "1010",
          'B': "1011",
          'C': "1100",
          'D': "1101",
          'E': "1110",
          'F': "1111"}
    bin = ""
    for i in range(len(s)):
        bin = bin + mp[s[i]]
    return bin

def hexToBinary(hexdigits):
	binarydigits = ""
	for hexdigit in hexdigits:
		binarydigits += bin(int(hexdigit,16))[2:].zfill(4)
	return binarydigits

#USING TWO FUNCTIONS TO PERMUTATE JUST TO VERIFY 
def permute(k, arr, n):
    permutation = ""
    for i in range(0, n):
        permutation = permutation + k[arr[i] - 1]
    return permutation
def lookup_in_sbox(bits, sbox):
    row = int(bits[0] + bits[3], 2)
    col = int(bits[1] + bits[2], 2)
    return '{0:02b}'.format(sbox[row][col])
    

def left_half(bits):
    return bits[:int((len(bits)/2))]

def right_half(bits):
    return bits[int((len(bits)/2)):]
    
def xor(a, b):
    ans = ""
    for i in range(len(a)):
        if a[i] == b[i]:
            ans = ans + "0"
        else:
            ans = ans + "1"
    return ans
 



### Multiple Columnar Transposition Technique

In [2]:
plaintext = "wearediscovered"
keyword = "ZEBRAS"

ciphertext = encrypt(plaintext, keyword)
print("Fist Round: "  + ciphertext)
ciphertext = encrypt(ciphertext, keyword)
print("Second Round: "  + ciphertext)

Fist Round: evacdeserodewir
Second Round: ddarrveicoeeesw


### 2) Shift Row Transformation

In [3]:
# Example usage:
matrix = createEncMatrix(4, ciphertext)
print("Input Matrix:\n ")
for row in matrix:
    print(row)
# Perform a row shift
shift_rows(matrix)

# Print the modified matrix
print("\nInput Shift Row Matrix:\n ")
for row in matrix:
    print(row)

flat_matrix = [element for row in matrix for element in row]
result = ''.join(flat_matrix)
print("\n64-bit output: " + result)

# Example usage:
chunks = split_string_into_8_byte_chunks_with_padding(result)
print("\nDivide output in 8-bit blocks ")
print(chunks)
print("\nFirst block: " + chunks[0])

Input Matrix:
 
['d', 'd', 'a', 'r']
['r', 'v', 'e', 'i']
['c', 'o', 'e', 'e']
['e', 's', 'w']

Input Shift Row Matrix:
 
['d', 'd', 'a', 'r']
['v', 'e', 'i', 'r']
['e', 'e', 'c', 'o']
['e', 's', 'w']

64-bit output: ddarveireecoesw

Divide output in 8-bit blocks 
['d', 'd', 'a', 'r', 'v', 'e', 'i', 'r', 'e', 'e', 'c', 'o', 'e', 's', 'w']

First block: d


### 3) SUBKEYS K1 & K2 GENERATION

In [4]:
perm_p10 =  [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
perm_p8 =  [6, 3, 7, 4, 8, 5, 10, 9]

key = "k"
key_chunks = split_string_into_8_byte_chunks_with_padding(key)

hex_key = "".join(hex(ord(x))[2:] for x in key_chunks[0])
binary_key = hexToBinary(hex_key)

#Padd the key if its not 10 bits
if len(binary_key) < 10:
    for i in range(10 - len(binary_key)):
        binary_key = '0' + binary_key  # Pad with leading zeros

print("10-bit key: " + key)
print("Key hexadecimal: " + hex_key)
print("Key in binary: " + binary_key)


#Permutate the key
key_perm = permute(binary_key, perm_p10, 10)
print("Key permutation 10: " + key_perm)

#Split key into two halves
left_key = key_perm[0:5]
right_key = key_perm[5:10]

10-bit key: k
Key hexadecimal: 6b
Key in binary: 0001101011
Key permutation 10: 0101110100


#### 3.1) Perfom Circular Left Shift of 1 on both keys

In [5]:
print("\n\tBefore Circular Shift\n\nLS-1 (L): " +left_key + "\nLS-1 (R): " + right_key)

left_key_S1 = shift_left(left_key, 1)
right_key_S1 = shift_left(right_key, 1)

print("\n\tCircular Left Shift 1 - 1\nLS-1 (L): " +left_key_S1 + "\nLS-1 (R): " + right_key_S1)

K_LF1_result = left_key_S1 + right_key_S1

#Perform p8 on the 10 bits
subkey_1 = permute(K_LF1_result, perm_p8, 8)
print("\nSub-key 1: " + subkey_1)

#Go back to initial LS1 halves and perform a circular left shift 2 on both halves
left_key_S2 = shift_left(left_key_S1, 2)
right_key_S2 = shift_left(right_key_S1, 2)

print("\n\tCircular Left Shift - 2\nLS-2 (L): " + left_key_S2 + "\nLS-2 (R): " + right_key_S2)
K_LF2_result = left_key_S2 + right_key_S2
subkey_2 = permute(K_LF2_result, perm_p8, 8)
print("\nSub-key 2: " + subkey_2)


	Before Circular Shift

LS-1 (L): 01011
LS-1 (R): 10100

	Circular Left Shift 1 - 1
LS-1 (L): 10110
LS-1 (R): 01001

Sub-key 1: 01110010

	Circular Left Shift - 2
LS-2 (L): 11010
LS-2 (R): 00101

Sub-key 2: 00011010


### Now that both keys have been generated, continue with the S-DES IP

In [6]:
#Initial Permutation only inputs binary, so, convert string to hex and then to binary

#HEX BLOCK 1
hex_block = "".join(hex(ord(x))[2:] for x in chunks[0])
hex_block = ""+ hex_block
print("Block being encrypted: " + chunks[0])
print("Block in hexadecimal: " + hex_block)

#BINARY
bin_block = hexToBinary(hex_block)
print("Block in binary: " + bin_block)


Block being encrypted: d
Block in hexadecimal: 64
Block in binary: 01100100


### 4) Initial Permutation

In [7]:
initial_perm =  [2, 6, 3, 1, 4, 8, 5, 7]

ip = permute(bin_block, initial_perm, 8)
print("Block to-be permuted: " + bin_block)
print("Initial Permutation (IP): ", ip)
        


Block to-be permuted: 01100100
Initial Permutation (IP):  11100000


### 5) F_k Function

In [10]:
def f_k(bits, subkey):
    left_bits = left_half(bits)
    right_bits = right_half(bits)
    
    out_bits = permute(right_bits, expansion_table, 8)
    print("\nPermute expansion table: " + out_bits)
    out_bits = xor(out_bits, subkey)
    print("XOR operation: " + out_bits)
    
    out_bits = lookup_in_sbox(left_half(out_bits), S0) + lookup_in_sbox(right_half(out_bits), S1)
    print("S-Box: " + out_bits)
    out_bits = permute(out_bits, perm_p4, 4)
    print("P4 permutation: " + out_bits)
    
    return xor(left_bits, out_bits)

expansion_table = [4, 1, 2, 3, 2, 3, 4, 1]
perm_p4 = [2, 4, 3, 1]

S0 = [[1, 0, 3, 2],
      [3, 2, 1, 0],
      [0, 2, 1, 3],
      [3, 1, 3, 2]]

S1 = [[0, 1, 2, 3],
      [2, 0, 1, 3],
      [3, 0, 1, 0],
      [2, 1, 0, 3]]


left_ip = left_half(ip)
print("\nLeft of IP: " + left_ip)
right_ip = right_half(ip)
print("Right of IP: " + right_ip)

fk_1 = f_k(ip, subkey_1)
print("\nF_k function: " + fk_1)


Left of IP: 1110
Right of IP: 0000

Permute expansion table: 00000000
XOR operation: 01110010
S-Box: 0001
P4 permutation: 0100

F_k function: 1010


### 6) The Switch Function

In [12]:
def switch(leftbits, rightbits):
    return rightbits + leftbits

switched_bits = switch(fk_1, right_half(ip))
print("Switch Function: " + switched_bits)

fk_2 = f_k(switched_bits, subkey_2)
print("\nF_k function: " + fk_2)

print("Output: " + fk_2 + fk_1)

Switch Function: 00001010

Permute expansion table: 01010101
XOR operation: 01001111
S-Box: 1111
P4 permutation: 1111

F_k function: 1111
Output: 11111010


### 8) Inverse Permutation

In [15]:
inverse_perm = [4, 1, 3, 5, 7, 2, 8, 6]

fk_final = fk_2 + fk_1
ciphertext = permute(fk_final, inverse_perm, 8)
print("Ciphertext: " + ciphertext)

Ciphertext: 11111100
